In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization, GlobalAveragePooling2D, LSTM, TimeDistributed
from tensorflow.keras.applications import InceptionV3, VGG16, VGG19, ResNet50, DenseNet201, MobileNet
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
import splitfolders
source_folder = './DATASET'


output_folder = './DATASET/split_data2'


splitfolders.ratio(source_folder, output=output_folder, ratio=(0.7, 0.3))

print("Data splitting completed.")

Copying files: 91 files [00:01, 82.13 files/s]

Data splitting completed.


In [13]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    strategy = tf.distribute.OneDeviceStrategy("/device:GPU:0")  # Use the first available GPU (T4 GPU)
else:
    strategy = tf.distribute.OneDeviceStrategy("/device:CPU:0")

In [14]:
with strategy.scope():
    train_datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)

    train_dir = './DATASET/split_data2/train'
    test_dir = './DATASET/split_data2/val'

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',  # Multi-class classification
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',  # Multi-class classification
    )

Found 62 images belonging to 4 classes.
Found 29 images belonging to 4 classes.


In [15]:
# InceptionV3 Model

def InceptionV3Model():
  base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"InceptionV3 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"InceptionV3 Test Accuracy: {test_accuracy * 100:.2f}%")


In [16]:
# ResNet50 Model

def ResNet50Model():
  base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
  for layer in base_model.layers:
    layer.trainable = False
  x = base_model.output
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(4, activation='softmax')(x)  # Four classes: Stage 1, Stage 2, Stage 3, No ROP
  model = models.Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  r = model.fit_generator(
  train_generator,
  validation_data=test_generator,
  epochs=10,
  steps_per_epoch=len(train_generator),
  validation_steps=len(test_generator))
  test_loss, test_accuracy = model.evaluate(test_generator)
  train_loss, train_accuracy = model.evaluate(train_generator)
  print(f"ResNet50 Train Accuracy: {train_accuracy * 100:.2f}%")
  print(f"ResNet50 Test Accuracy: {test_accuracy * 100:.2f}%")

In [17]:
# Define the MobileNet model
def MobileNetModel():
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(28, activation='relu')(x)
    output = Dense(5, activation='softmax')(x)  # Three classes: Stage_1, Stage_2, Stage_3
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    history = model.fit_generator(
        train_generator,
        validation_data=test_generator,
        epochs=10,
        steps_per_epoch=len(train_generator),
        validation_steps=len(test_generator)
    )
    test_loss, test_accuracy = model.evaluate(test_generator)
    train_loss, train_accuracy = model.evaluate(train_generator)
    print(f"MobileNet Train Accuracy: {train_accuracy * 100:.2f}%")
    print(f"MobileNet Test Accuracy: {test_accuracy * 100:.2f}%")

# Call the MobileNetModel function
MobileNetModel()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128 

C:\Users\Rumman Fardeen\AppData\Local\Temp\ipykernel_4436\1773810525.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10



InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 215, in start

  File "c:\Python311\Lib\asyncio\base_events.py", line 604, in run_forever

  File "c:\Python311\Lib\asyncio\base_events.py", line 1909, in _run_once

  File "c:\Python311\Lib\asyncio\events.py", line 80, in _run


In [18]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './DATASET/split_data2/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    './DATASET/split_data2/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 62 images belonging to 4 classes.
Found 29 images belonging to 4 classes.


In [20]:
# InceptionV3 Model
def InceptionV3Model(train_generator, test_generator):
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(5, activation='softmax')(x)  # Three classes: Stage 1, Stage 2, Stage 3
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=Adam(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    r = model.fit_generator(
        train_generator,
        validation_data=test_generator,
        epochs=10,
        steps_per_epoch=len(train_generator),
        validation_steps=len(test_generator))
    test_loss, test_accuracy = model.evaluate(test_generator)
    train_loss, train_accuracy = model.evaluate(train_generator)
    print(f"InceptionV3 Train Accuracy: {train_accuracy * 100:.2f}%")
    print(f"InceptionV3 Test Accuracy: {test_accuracy * 100:.2f}%")

# Call the function
InceptionV3Model(train_generator, test_generator)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 111, 111, 32)         864       ['input_4[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 111, 111, 32)         96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 111, 111, 32)         0         ['batch_normalization_94

C:\Users\Rumman Fardeen\AppData\Local\Temp\ipykernel_4436\2694609193.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 215, in start

  File "c:\Python311\Lib\asyncio\base_events.py", line 604, in run_forever

  File "c:\Python311\Lib\asyncio\base_events.py", line 1909, in _run_once

  File "c:\Python311\Lib\asyncio\events.py", line 80, in _run


In [21]:
# Define ResNet50 Model function
def ResNet50Model(train_generator, test_generator):
    # Load pre-trained ResNet50 model
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Freeze pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Add custom dense layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(3, activation='softmax')(x)  # Three classes: Stage 1, Stage 2, Stage 3
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compile the model
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=10,
        steps_per_epoch=len(train_generator),
        validation_steps=len(test_generator)
    )
    
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(test_generator)
    train_loss, train_accuracy = model.evaluate(train_generator)
    print(f"ResNet50 Train Accuracy: {train_accuracy * 100:.2f}%")
    print(f"ResNet50 Test Accuracy: {test_accuracy * 100:.2f}%")

# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './DATASET/split_data2/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    './DATASET/split_data2/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# Call ResNet50Model function
ResNet50Model(train_generator, test_generator)

Found 62 images belonging to 4 classes.
Found 29 images belonging to 4 classes.


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 215, in start

  File "c:\Python311\Lib\asyncio\base_events.py", line 604, in run_forever

  File "c:\Python311\Lib\asyncio\base_events.py", line 1909, in _run_once

  File "c:\Python311\Lib\asyncio\events.py", line 80, in _run


In [24]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

def VGG16Model(train_generator, test_generator):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    
    x = Flatten()(base_model.output)
    x = Dense(13, activation='relu')(x)
    output = Dense(5, activation='softmax')(x)  # Three classes: Stage 1, Stage 2, Stage 3
    model = Model(inputs=base_model.input, outputs=output)
    
    model.compile(optimizer=Adam(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.summary()
    
    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=10,
        steps_per_epoch=len(train_generator),
        validation_steps=len(test_generator)
    )
    
    test_loss, test_accuracy = model.evaluate(test_generator)
    train_loss, train_accuracy = model.evaluate(train_generator)
    print(f"VGG16 Train Accuracy: {train_accuracy * 100:.2f}%")
    print(f"VGG16 Test Accuracy: {test_accuracy * 100:.2f}%")

# Example of how to call the function with the data generators
VGG16Model(train_generator, test_generator)


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 215, in start

  File "c:\Python311\Lib\asyncio\base_events.py", line 604, in run_forever

  File "c:\Python311\Lib\asyncio\base_events.py", line 1909, in _run_once

  File "c:\Python311\Lib\asyncio\events.py", line 80, in _run


In [25]:
# Define the InceptionV3 model within the strategy scope
with strategy.scope():
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation='relu')(x)
    output = Dense(5, activation='softmax')(x)  # 3 classes (Stage_1, Stage_2, Stage_3)

    model = Model(inputs=base_model.input, outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the available GPU (T4 GPU)
with strategy.scope():
    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=10,
        steps_per_epoch=len(train_generator),
        validation_steps=len(test_generator)
    )

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start

  File "C:\Users\Rumman Fardeen\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 215, in start

  File "c:\Python311\Lib\asyncio\base_events.py", line 604, in run_forever

  File "c:\Python311\Lib\asyncio\base_events.py", line 1909, in _run_once

  File "c:\Python311\Lib\asyncio\events.py", line 80, in _run
